In [1]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
base_url = "https://pilkada2017.kpu.go.id/pemilih/dpt/2/DKI%20JAKARTA/"
#for round 1: https://pilkada2017.kpu.go.id/pemilih/dpt/1/DKI%20JAKARTA/

In [3]:
r_prov = requests.get(base_url+"listDps.json").json()
kota_list = [str(i['namaKabKota']) for i in r_prov['aaData']]

In [4]:
kec_data = []
for kota in kota_list:
    url = base_url+kota.replace(" ","%20")+"/listDps.json"
    kec = requests.get(url).json()
    kec_data += ([d for d in kec['aaData']])

In [7]:
df = pd.DataFrame(kec_data)
df[['namaKecamatan','totalPemilih']].head(5)

,namaKecamatan,totalPemilih
0,CENGKARENG,349548
1,GROGOL PETAMBURAN,166892
2,KALIDERES,284007
3,KEBON JERUK,240281
4,KEMBANGAN,191879


In [8]:
#make URL for kelurahan-level data from kab-kec pairs
kec_pair = df[['namaKabKota','namaKecamatan']].to_dict(orient='records')
kec_url_list = []
for i in kec_pair:
    for k,v in i.items():
        i[k]=v.replace(" ","%20")
    kec_url_list.append(str(i['namaKabKota']+"/"+i['namaKecamatan']))

In [10]:
kel_data = []
for kec in kec_url_list:
    url = base_url+kec+"/listDps.json"
    kel = requests.get(url).json()
    kel_data += ([d for d in kel['aaData']])

In [11]:
df_kel = pd.DataFrame(kel_data)
df_kel.head(5)

,jmlDifabel1,jmlDifabel2,jmlDifabel3,jmlDifabel4,jmlDifabel5,jmlPemilihKosong,jmlPemilihLaki,jmlPemilihPemulaLaki,jmlPemilihPemulaPerempuan,jmlPemilihPerempuan,jmlTps,namaKabKota,namaKecamatan,namaKelurahan,namaPropinsi,persenDifabel,persenPemilihPemula,totalDifabel,totalPemilih,totalPemilihPemula
0,3,1,3,262,143,0,25399,74,63,25490,83,JAKARTA BARAT,CENGKARENG,CENGKARENG BARAT,DKI JAKARTA,0.81,0.27,412,50889,137
1,22,8,11,6,17,0,29532,95,108,29159,103,JAKARTA BARAT,CENGKARENG,CENGKARENG TIMUR,DKI JAKARTA,0.11,0.35,64,58691,203
2,9,1,2,3,2,0,30945,100,102,30559,100,JAKARTA BARAT,CENGKARENG,DURI KOSAMBI,DKI JAKARTA,0.03,0.33,17,61504,202
3,19,11,20,23,22,0,51727,160,182,49418,178,JAKARTA BARAT,CENGKARENG,KAPUK,DKI JAKARTA,0.09,0.34,95,101145,342
4,0,0,1,0,1,0,14155,35,32,13254,47,JAKARTA BARAT,CENGKARENG,KEDAUNG KALI ANGKE,DKI JAKARTA,0.01,0.24,2,27409,67


In [12]:
df_kel.to_csv("dpt2.csv")